<a href="https://colab.research.google.com/github/infurnex/GUVI_DATATHON/blob/main/Pravartak_Datathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import json
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
import matplotlib. pyplot as plt

In [2]:
df= pd.read_excel("/content/House_Rent_Train.xlsx")

In [3]:
df= df.dropna()

# Data Preprocessing

In [4]:
df.describe()

,latitude,longitude,gym,lift,swimming_pool,negotiable,property_size,property_age,bathroom,cup_board,floor,total_floor,balconies,rent
count,20377.000000,20377.000000,20377.000000,20377.000000,20377.000000,20377.000000,20377.000000,20377.000000,20377.000000,20377.000000,20377.000000,20377.000000,20377.000000,20377.000000
mean,12.945750,77.637238,0.226235,0.361142,0.174461,0.711488,1061.437209,5.777053,1.857732,4.567503,1.869510,3.798106,1.147863,19048.502969
std,0.029587,0.110758,0.418404,0.480344,0.379515,0.453081,591.857798,6.362450,0.710933,336.427053,2.181633,3.183849,0.993290,8433.444157
min,12.900004,77.500072,0.000000,0.000000,0.000000,0.000000,1.000000,-1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,8000.000000
25%,12.918232,77.571912,0.000000,0.000000,0.000000,0.000000,700.000000,1.000000,1.000000,1.000000,1.000000,2.000000,0.000000,12500.000000
50%,12.943792,77.633367,0.000000,0.000000,0.000000,1.000000,1044.000000,5.000000,2.000000,2.000000,1.000000,3.000000,1.000000,17500.000000
75%,12.970922,77.695692,0.000000,1.000000,0.000000,1.000000,1280.000000,10.000000,2.000000,3.000000,2.000000,4.000000,2.000000,24000.000000
max,12.999999,80.266346,1.000000,1.000000,1.000000,1.000000,50000.000000,400.000000,21.000000,48026.000000,25.000000,26.000000,13.000000,50000.000000


In [5]:
categorical= [ 'type', 'lease_type', 'facing', 'water_supply', 'building_type', 'furnishing', 'parking']

In [6]:
# Initialize the OneHotEncoder
encoder = LabelEncoder()

# Fit and transform the encoder on the categorical columns
df['type'] = encoder.fit_transform(df['type'])

df['type']

0        2
1        2
2        3
3        1
4        3
        ..
20550    9
20551    2
20552    2
20553    2
20554    1
Name: type, Length: 20555, dtype: int64

In [10]:
df['lease_type']= encoder.fit_transform(df['lease_type'])

In [12]:
df['facing']= encoder.fit_transform(df['facing'])

In [7]:
df['water_supply']= encoder.fit_transform(df['water_supply'])
df['building_type']= encoder.fit_transform(df['building_type'])
df['furnishing']= encoder.fit_transform(df['furnishing'])
df['parking']= encoder.fit_transform(df['parking'])

In [10]:
df.head()

,id,type,locality,activation_date,latitude,longitude,lease_type,gym,lift,swimming_pool,...,bathroom,facing,cup_board,floor,total_floor,amenities,water_supply,building_type,balconies,rent
1,ff8081815ee25e15015ee50004da2acd,2,Bellandur,2017-09-10 12:48:00,12.929557,77.672280,0,0,1,0,...,2.0,2,2.0,3.0,4.0,"{""LIFT"":true,""GYM"":false,""INTERNET"":false,""AC""...",1,0,2.0,22000.0
2,ff80818163f29c9e0163f46770b873e7,3,Thiruvanmiyur,2018-12-06 22:02:00,12.982870,80.262012,3,0,1,0,...,3.0,0,3.0,1.0,5.0,"{""LIFT"":true,""GYM"":false,""INTERNET"":false,""AC""...",2,0,3.0,28000.0
3,ff80818164b68a700164b74b1b247a1d,1,Attiguppe,20-07-2018 16:22,12.955991,77.531634,3,0,0,0,...,1.0,0,1.0,1.0,2.0,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",1,3,0.0,8000.0
4,ff80818163b1faf00163b4b9b8a163b1,3,Kodihalli,31-05-2018 17:41,12.963903,77.649446,3,0,0,0,...,3.0,0,4.0,0.0,0.0,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",1,3,1.0,45000.0
5,ff8081815f48e4aa015f4dd1a9c6696b,1,"Seetharampalya,Hoodi",24-10-2017 18:29,12.986196,77.718314,3,1,1,1,...,2.0,0,1.0,3.0,5.0,"{""LIFT"":true,""GYM"":true,""INTERNET"":false,""AC"":...",2,0,2.0,18000.0


In [11]:
# spliting the data into columns of their key value and converting into the int values from bool
amenities_dicts = [json.loads(entry) for entry in df["amenities"]]
data = pd.DataFrame(amenities_dicts)
mean_value= data.mean()
data= data.fillna(mean_value)
data= data.astype(int)

In [12]:
data

,LIFT,GYM,INTERNET,AC,CLUB,INTERCOM,POOL,CPA,FS,SERVANT,SECURITY,SC,GP,PARK,RWH,STP,HK,PB,VP
0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,1,1,0,1,1
1,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
4,1,1,0,0,1,0,1,1,1,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20372,1,0,1,0,0,0,0,0,1,0,1,1,1,1,0,0,1,0,1
20373,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0
20374,1,1,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0
20375,1,0,1,0,0,0,0,1,1,0,1,1,0,1,0,0,1,1,1


In [13]:
# adding the splitted data to df and removing amenities from data
df = df.join(data)
df = df.drop("amenities", axis = 1)
# removing locality due to highly related to longitude and latitude
df= df.drop('locality', axis=1)
df= df.drop('activation_date', axis=1)

In [14]:
df= df.drop('id', axis=1)

In [ ]:
df.info()

In [16]:
# droping the values which are added twice
df=df.drop('GYM', axis=1)
df=df.drop('LIFT', axis=1)

In [17]:
# defining new factor which is more co-related to rent values
df['basic']= df['bathroom']*df['property_size']*df['lift']
df['mid']= df['PARK']*df['gym']*df['INTERNET']
df['location']=df['longitude']*df['latitude']*df['property_size']

In [18]:
df['neg']= df['parking']*df['building_type']

In [19]:
df= df.drop('neg', axis=1)

In [32]:
# finding co-relation between the rent and the factors
corr_matrix= df.corr()
corr_matrix['rent'].sort_values(ascending= False)

rent             1.000000
bathroom         0.677367
location         0.587464
property_size    0.587153
basic            0.577313
balconies        0.472521
lift             0.461769
total_floor      0.436403
swimming_pool    0.434065
gym              0.431096
type             0.344673
floor            0.305168
mid              0.250324
longitude        0.163305
lease_type       0.076220
facing           0.067047
negotiable       0.062798
latitude         0.033017
PB               0.007522
PARK             0.006613
CPA              0.005609
FS               0.004560
VP               0.002943
INTERNET         0.002732
SC               0.002297
CLUB             0.002234
RWH              0.002011
GP               0.001169
STP             -0.000160
cup_board       -0.001259
HK              -0.002493
POOL            -0.003108
INTERCOM        -0.003460
SECURITY        -0.004826
AC              -0.005907
SERVANT         -0.006429
property_age    -0.011020
water_supply    -0.014957
furnishing  

#Visualization

In [ ]:
# we can use vif for checking the which values are more co-related to eachother
X = df[['type', 'latitude', 'longitude','lease_type','gym','lift','swimming_pool','negotiable','furnishing','parking','property_age','property_size','bathroom','facing','cup_board','floor','total_floor','water_supply','building_type','balconies','rent']]
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]

print(vif_data)

In [ ]:
# Find correlation matrix
correlation_matrix = df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, cmap="coolwarm", center=0)
plt.title("Correlation Matrix Heatmap")
plt.show()



In [ ]:
correlation_with_rent = df.corrwith(df['rent'])

# Print the correlation coefficients
print(correlation_with_rent)

correlation_with_rent.plot(kind='bar')
plt.title("Correlation of Features with Rent")
plt.xlabel("Features")
plt.ylabel("Correlation Coefficient")
plt.show()

#Splitting

In [24]:
df['balconies'].value_counts()

1.0     7957
0.0     5868
2.0     4634
3.0     1582
4.0      288
5.0       40
6.0        4
10.0       3
13.0       1
Name: balconies, dtype: int64

In [59]:
#value counts
#type                lease_type        gym
#2    11607          1    10113        0  15767
#3     4368          2     9622        1  4610
#1     4140          3      584
#4      262          4       58

In [25]:
y= df['rent']

In [26]:
X= df.drop('rent',axis=1)

In [45]:
predictors= ['type', 'latitude', 'longitude', 'lift', 'swimming_pool', 'furnishing', 'parking', 'property_size', 'bathroom', 'facing', 'total_floor', 'water_supply', 'INTERNET', 'AC','INTERCOM', 'SECURITY']

In [46]:
# Initialise and Prepare data
X_train, X_cv, y_train, y_cv = train_test_split(X[predictors], y, test_size=0.2, stratify=df['type'], random_state=42)

In [33]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20377 entries, 1 to 20531
Data columns (total 40 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   type           20377 non-null  int64  
 1   latitude       20377 non-null  float64
 2   longitude      20377 non-null  float64
 3   lease_type     20377 non-null  int64  
 4   gym            20377 non-null  int64  
 5   lift           20377 non-null  int64  
 6   swimming_pool  20377 non-null  int64  
 7   negotiable     20377 non-null  int64  
 8   furnishing     20377 non-null  int64  
 9   parking        20377 non-null  int64  
 10  property_size  20377 non-null  int64  
 11  property_age   20377 non-null  float64
 12  bathroom       20377 non-null  float64
 13  facing         20377 non-null  int64  
 14  cup_board      20377 non-null  float64
 15  floor          20377 non-null  float64
 16  total_floor    20377 non-null  float64
 17  water_supply   20377 non-null  int64  
 18  buildi

#Model

In [ ]:
# Initialize an XGBoost Regressor model
model = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.25, random_state=42)

# Fit the model to the training data
model.fit(X_train, y_train)

In [ ]:
# Make predictions using the trained model on the cross-validation set and Print
y_pred = model.predict(X_cv)
mse = mean_squared_error(y_cv, y_pred)
print(f"Mean Squared Error: {mse}")

In [ ]:
#Random Grid Search
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_grid = {
    'n_estimators': np.arange(50, 501, 50),
    'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],
    'max_depth': np.arange(3, 11),
    'min_child_weight': np.arange(1, 11),
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],
    'reg_alpha': [0, 0.1, 0.5, 1.0],
    # 'reg_lambda': [0, 0.1, 0.5, 1.0]
}

In [ ]:
xgb_regressor = xgb.XGBRegressor(random_state=42, objective='reg:squarederror')
random_search = RandomizedSearchCV(
    xgb_regressor,
    param_distributions=param_grid,
    n_iter=20,  # Number of random combinations to try
    scoring='neg_mean_squared_error',  # Evaluation metric
    cv=5,  # Number of cross-validation folds
    verbose=1,
    n_jobs=-1,  # Use all available CPU cores
    random_state=42
)

random_search.fit(X_train, y_train)

In [116]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'subsample': 0.9, 'reg_alpha': 1.0, 'n_estimators': 450, 'min_child_weight': 6, 'max_depth': 4, 'learning_rate': 0.1, 'gamma': 0.3, 'colsample_bytree': 0.6}
Best Score: -12454422.590882456


In [47]:
model = xgb.XGBRegressor(n_estimators=450, max_depth=4, learning_rate=0.1,reg_alpha=1.0, colsample_bytree= 0.6, min_child_weight=6, subsample=0.9,gamma=0.3, random_state=42)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.3, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=6, missing=nan, monotone_constraints=None,
             n_estimators=450, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [48]:
# Calculate Root Mean Squared Error
y_pred = model.predict(X_cv)
mse = mean_squared_error(y_cv, y_pred)
rmse= mse**0.5
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 3820.062476984353


#Test

##Test Data_processing

In [210]:
# Read the test data and initialise
test= pd.read_csv('/content/House_Rent_Test - test.csv.csv')

In [211]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               4500 non-null   object 
 1   type             4500 non-null   object 
 2   locality         4500 non-null   object 
 3   activation_date  4500 non-null   object 
 4   latitude         4500 non-null   float64
 5   longitude        4500 non-null   float64
 6   lease_type       4500 non-null   object 
 7   gym              4500 non-null   int64  
 8   lift             4500 non-null   int64  
 9   swimming_pool    4500 non-null   int64  
 10  negotiable       4500 non-null   int64  
 11  furnishing       4500 non-null   object 
 12  parking          4500 non-null   object 
 13  property_size    4500 non-null   int64  
 14  property_age     4500 non-null   int64  
 15  bathroom         4500 non-null   int64  
 16  facing           4500 non-null   object 
 17  cup_board     

In [212]:
test

,id,type,locality,activation_date,latitude,longitude,lease_type,gym,lift,swimming_pool,...,property_age,bathroom,facing,cup_board,floor,total_floor,amenities,water_supply,building_type,balconies
0,ff8081815df539bc015df947ce976cca,BHK2,Basavanagudi,22-08-2017 09:00,12.941603,77.568156,FAMILY,0,1,0,...,1,2,N,2,3,3,"{""LIFT"":true,""GYM"":false,""INTERNET"":false,""AC""...",CORP_BORE,AP,1
1,ff80818157288e9301572c05651853a6,BHK2,Rajaji Nagar,17-09-2017 16:33,12.998803,77.561887,ANYONE,0,0,0,...,7,1,S,2,0,3,"{""LIFT"":false,""GYM"":false,""INTERNET"":true,""AC""...",CORPORATION,IF,0
2,ff8081815f1afc58015f1b831fde166e,BHK1,Jeevan Bima Nagar,14-04-2018 17:27,12.966467,77.661063,ANYONE,0,0,0,...,10,1,S,1,0,1,"{""LIFT"":false,""GYM"":false,""INTERNET"":true,""AC""...",CORPORATION,IF,0
3,ff8081816035b31901603f7922c046e0,BHK1,Jayanagar 1st Block,11-12-2017 13:15,12.941533,77.592606,ANYONE,0,0,0,...,8,1,E,0,2,2,"{""LIFT"":false,""GYM"":false,""INTERNET"":true,""AC""...",CORP_BORE,IF,0
4,ff8081815c3582f5015c38fc31aa73a4,BHK3,Whitefield Hope Farm Junction,11-07-2017 16:44,12.971083,77.751625,ANYONE,1,1,1,...,0,3,E,3,3,4,"{""LIFT"":true,""GYM"":true,""INTERNET"":true,""AC"":f...",BOREWELL,AP,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,ff8081815216f51101521aa9d46c191f,BHK2,Hanumanthnagar,10-06-2018 18:22,12.941654,77.559999,FAMILY,0,0,0,...,2,2,N,2,3,3,"{""LIFT"":false,""GYM"":false,""INTERNET"":true,""AC""...",CORP_BORE,IF,2
4496,ff8081816098166901609bf52bb702a8,BHK2,Lakkasandra,21-01-2018 14:36,12.942973,77.597738,ANYONE,0,0,0,...,10,1,N,2,2,3,"{""LIFT"":false,""GYM"":false,""INTERNET"":true,""AC""...",CORPORATION,IF,1
4497,ff808181629ac9860162a7ac1a4253bb,BHK1,"Jeevan Samskruthi Apartment, Shiridi Sai Templ...",30-05-2018 16:10,12.949315,77.713807,ANYONE,0,0,0,...,10,1,W,1,2,3,"{""SC"":false,""INTERCOM"":false,""AC"":false,""PB"":f...",CORP_BORE,IH,0
4498,ff8081815b57c717015b580af22f1bbd,BHK3,Mullur,11-07-2018 16:17,12.914104,77.716428,FAMILY,1,1,1,...,1,3,N,4,10,10,"{""SC"":false,""INTERCOM"":true,""AC"":false,""PB"":tr...",BOREWELL,AP,3


In [215]:
encoder = LabelEncoder()

# Fit and transform the encoder on the categorical columns
test['type'] = encoder.fit_transform(test['type'])
test['lease_type']= encoder.fit_transform(test['lease_type'])
test['facing']= encoder.fit_transform(test['facing'])
test['water_supply']= encoder.fit_transform(test['water_supply'])
test['building_type']= encoder.fit_transform(test['building_type'])
test['furnishing']= encoder.fit_transform(test['furnishing'])
test['parking']= encoder.fit_transform(test['parking'])

In [216]:
test.head()

,type,locality,activation_date,latitude,longitude,lease_type,gym,lift,swimming_pool,negotiable,...,property_age,bathroom,facing,cup_board,floor,total_floor,amenities,water_supply,building_type,balconies
0,1,Basavanagudi,22-08-2017 09:00,12.941603,77.568156,3,0,1,0,0,...,1,2,1,2,3,3,"{""LIFT"":true,""GYM"":false,""INTERNET"":false,""AC""...",2,0,1
1,1,Rajaji Nagar,17-09-2017 16:33,12.998803,77.561887,0,0,0,0,1,...,7,1,4,2,0,3,"{""LIFT"":false,""GYM"":false,""INTERNET"":true,""AC""...",1,2,0
2,0,Jeevan Bima Nagar,14-04-2018 17:27,12.966467,77.661063,0,0,0,0,1,...,10,1,4,1,0,1,"{""LIFT"":false,""GYM"":false,""INTERNET"":true,""AC""...",1,2,0
3,0,Jayanagar 1st Block,11-12-2017 13:15,12.941533,77.592606,0,0,0,0,0,...,8,1,0,0,2,2,"{""LIFT"":false,""GYM"":false,""INTERNET"":true,""AC""...",2,2,0
4,2,Whitefield Hope Farm Junction,11-07-2017 16:44,12.971083,77.751625,0,1,1,1,1,...,0,3,0,3,3,4,"{""LIFT"":true,""GYM"":true,""INTERNET"":true,""AC"":f...",0,0,1


In [217]:
amenities_dicts = [json.loads(entry) for entry in test["amenities"]]
data = pd.DataFrame(amenities_dicts)
mean_value= data.mean()
data= data.fillna(mean_value)
data= data.astype(int)

In [219]:
data

,LIFT,GYM,INTERNET,AC,CLUB,INTERCOM,POOL,CPA,FS,SERVANT,SECURITY,SC,GP,PARK,RWH,STP,HK,PB,VP
0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0
3,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,1,1,1,0,1,1,1,1,1,0,1,0,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
4496,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4497,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4498,1,1,1,0,0,1,1,0,1,0,1,0,0,1,0,0,0,1,0


In [220]:
test = test.join(data)
test = test.drop("amenities", axis = 1)
test= test.drop('locality', axis=1)
test= test.drop('activation_date', axis=1)

In [ ]:
id= test['id']
test=test.drop('id', axis=1)

In [222]:
test=test.drop('GYM', axis=1)
test=test.drop('LIFT', axis=1)

In [223]:
test['basic']= test['bathroom']*test['property_size']*test['lift']
test['mid']= test['PARK']*test['gym']*test['INTERNET']
test['location']=test['longitude']*test['latitude']*test['property_size']

##Prediction

In [269]:
target = model.predict(test)

In [268]:
rents = pd.DataFrame({'id': id, 'rent': target})

In [267]:
rents.to_csv('rents.csv', index= False)



#Pickling & API

In [4]:
import pickle
model_filename = 'rent_predictor.pkl'

# Save the model to a .pkl file
with open(model_filename, 'wb') as model_file:
    pickle.dump(model, model_file)

NameError: ignored